In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [3]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [4]:
from torch.utils.data import Dataset
import os
import csv
from torch.utils.data import Dataset, DataLoader

class QuotesDataset(Dataset):
    def __init__(self, quotes_dataset_path):
        super().__init__()

        quotes_path = quotes_dataset_path

        self.quotes_list = []
        self.end_of_text_token = ""  # Add an end of text token if needed
        
        with open(quotes_path, newline='', encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            header = next(csv_reader)  # Skip the header row
            
            for row in csv_reader:
                quote_str = f"QUOTE: {row[0]} - {row[1]}, Category: {row[2]}{self.end_of_text_token}"
                self.quotes_list.append(quote_str)
        
    def __len__(self):
        return len(self.quotes_list)

    def __getitem__(self, item):
        return self.quotes_list[item]




In [5]:
dataset = QuotesDataset(quotes_dataset_path='quotes_data/quotes.csv')
quote_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [6]:
from transformers import AdamW, get_linear_schedule_with_warmup

BATCH_SIZE = 8
MAX_SEQ_LEN = 128
EPOCHS = 3
WARMUP_STEPS = 1000
LEARNING_RATE = 5e-5
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)

proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_quote_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch} started" + '=' * 30)

    for idx, quote in enumerate(quote_loader):

        #################### "Fit as many quote sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        quote_encoded = tokenizer.encode(quote[0], max_length=MAX_SEQ_LEN, truncation=True)
        quote_tens = torch.tensor(quote_encoded).unsqueeze(0).to(device)
        # Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if quote_tens.size()[1] > MAX_SEQ_LEN:
            continue

        # The first quote sequence in the sequence
        if not torch.is_tensor(tmp_quote_tens):
            tmp_quotes_tens = quote_tens
            continue
        else:
            # The next quote does not fit in so we process the sequence and leave the last quote
            # as the start for next sequence
            if tmp_quotes_tens.size()[1] + quote_tens.size()[1] > MAX_SEQ_LEN:
                work_quotes_tens = tmp_quotes_tens
                tmp_quotes_tens = quote_tens
            else:
                # Add the quote to sequence, continue and try to add more
                tmp_quotes_tens = torch.cat([tmp_quotes_tens, quote_tens[:, 1:]], dim=1)
                continue
        ################## Sequence ready, process it through the model ##################

        outputs = model(work_quotes_tens, labels=work_quotes_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_quoter_{epoch}.pt"))


EPOCH 0 started==============================
EPOCH 1 started==============================
EPOCH 2 started==============================


In [9]:
MODEL_EPOCH = 1

models_folder = "trained_models"

model_path = os.path.join(models_folder, f"gpt2_medium_quoter_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

quotes_output_file_path = f'generated_{MODEL_EPOCH}.quotes'

model.eval()
if os.path.exists(quotes_output_file_path):
    os.remove(quotes_output_file_path)
    
quote_num = 0
with torch.no_grad():
   
        for quote_idx in range(3):
        
            quote_finished = False

            cur_ids = torch.tensor(tokenizer.encode("quote:")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    quote_finished = True
                    break

            
            if quote_finished:
                
                quote_num = quote_num + 1
                
                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open(quotes_output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")

KeyboardInterrupt: 